# Visualising the network of 360 Giving grants

## Preparing the data

This notebook takes you through the process of preparing grants data
to produce a network diagram showing relationships between funders.
In this notebook I will:

1. Fetch grants data from [GrantNav](http://grantnav.threesixtygiving.org/)
2. Transform it into a tables of links between funders and recipients
3. Export those tables to the right format

The resulting table shows which funders share recipients in common with
other funders.

The first step is to import the [pandas](https://pandas.pydata.org/) data 
analysis library. I'll be using this to fetch and transform the data.

In [2]:
import pandas as pd

### 1. Fetch grants data

First I make a variable containing a URL to a GrantNav CSV file.
This link was found by performing a search on GrantNav and then
right clicking on the "CSV" download button in the top right, 
and selecting "Copy link location" (the exact way of copying the
link may vary in different browsers).

In [3]:
grantnav_url = 'http://grantnav.threesixtygiving.org/search.csv?json_query=%7B%22aggs%22%3A+%7B%22fundingOrganization%22%3A+%7B%22terms%22%3A+%7B%22field%22%3A+%22fundingOrganization.id_and_name%22%2C+%22size%22%3A+3%7D%7D%2C+%22recipientRegionName%22%3A+%7B%22terms%22%3A+%7B%22field%22%3A+%22recipientRegionName%22%2C+%22size%22%3A+3%7D%7D%2C+%22currency%22%3A+%7B%22terms%22%3A+%7B%22field%22%3A+%22currency%22%2C+%22size%22%3A+3%7D%7D%2C+%22recipientDistrictName%22%3A+%7B%22terms%22%3A+%7B%22field%22%3A+%22recipientDistrictName%22%2C+%22size%22%3A+3%7D%7D%2C+%22recipientOrganization%22%3A+%7B%22terms%22%3A+%7B%22field%22%3A+%22recipientOrganization.id_and_name%22%2C+%22size%22%3A+3%7D%7D%7D%2C+%22query%22%3A+%7B%22bool%22%3A+%7B%22filter%22%3A+%5B%7B%22bool%22%3A+%7B%22should%22%3A+%5B%5D%7D%7D%2C+%7B%22bool%22%3A+%7B%22should%22%3A+%5B%5D%7D%7D%2C+%7B%22bool%22%3A+%7B%22should%22%3A+%5B%5D%2C+%22must%22%3A+%7B%7D%7D%7D%2C+%7B%22bool%22%3A+%7B%22should%22%3A+%7B%22range%22%3A+%7B%22amountAwarded%22%3A+%7B%7D%7D%7D%2C+%22must%22%3A+%7B%7D%7D%7D%2C+%7B%22bool%22%3A+%7B%22should%22%3A+%5B%5D%7D%7D%2C+%7B%22bool%22%3A+%7B%22should%22%3A+%5B%5D%7D%7D%2C+%7B%22bool%22%3A+%7B%22should%22%3A+%5B%5D%7D%7D%2C+%7B%22bool%22%3A+%7B%22should%22%3A+%5B%5D%7D%7D%5D%2C+%22must%22%3A+%7B%22query_string%22%3A+%7B%22default_field%22%3A+%22_all%22%2C+%22query%22%3A+%22awardDate%3A%5B2016-01-01+TO+2017-12-31%5D%22%7D%7D%7D%7D%2C+%22extra_context%22%3A+%7B%22amountAwardedFixed_facet_size%22%3A+3%2C+%22awardYear_facet_size%22%3A+3%7D%2C+%22sort%22%3A+%7B%22_score%22%3A+%7B%22order%22%3A+%22desc%22%7D%7D%7D'

Make a list of the columns I want to use in the data - only a small number are relevant for this exercise.

In [4]:
columns = [
    "Identifier", "Currency", "Amount Awarded", "Award Date",
    "Recipient Org:Identifier", "Recipient Org:Name", 
    "Recipient Org:Charity Number", "Recipient Org:Company Number",
    "Funding Org:Identifier", "Funding Org:Name"
]

Then I use pandas to fetch the data. The `read_csv` method accepts an
`index_col` parameter which tells it which column to use as an index, while
passing our columns to `usecols` means only those columns will be returned.

This can take a little while to run as it downloads a large amount of data.

In [6]:
grants = pd.read_csv(grantnav_url, index_col='Identifier', usecols=columns)

I've added a dummy `Grants` variable with a value of 1 for each row. This
will help later when I want to count the number of grants.

In [7]:
grants.loc[:, "Grants"] = 1

Some funders put their name with spaces at the end, so I need to get rid of them.

In [8]:
grants.loc[:, "Funding Org:Name"] = grants["Funding Org:Name"].str.strip()

Turn the date field into a date format.

In [15]:
grants.loc[:, "Award Date"] = grants["Award Date"].astype('datetime64[ns]')

Let's take a look at the resulting data - first see how many rows there are:

In [16]:
len(grants)

60926

Then preview the list itself

In [17]:
grants

,Currency,Amount Awarded,Award Date,Recipient Org:Identifier,Recipient Org:Name,Recipient Org:Charity Number,Recipient Org:Company Number,Funding Org:Identifier,Funding Org:Name,Grants
Identifier,,,,,,,,,,
360G-SomersetCF-A397976,GBP,1470.00,2016-03-23,GB-CHC-1141319,Young People Frome,1141319,NaN,GB-COH-04530979,Somerset Community Foundation,1
360G-SomersetCF-A397978,GBP,2000.00,2016-04-04,GB-COH-09284683,GoCreate Taunton CIC,NaN,09284683,GB-COH-04530979,Somerset Community Foundation,1
360G-SomersetCF-A401718,GBP,2500.00,2016-06-24,360G-SomersetCF-ACC351246,St Francis Youth Club,NaN,NaN,GB-COH-04530979,Somerset Community Foundation,1
360G-SomersetCF-A404393,GBP,500.00,2016-06-30,360G-SomersetCF-ACC763854,Yeovil & Sherborne Hockey Club,NaN,NaN,GB-COH-04530979,Somerset Community Foundation,1
360G-SomersetCF-A404576,GBP,500.00,2016-06-07,GB-CHC-270392,Ashill Village Hall Committee,270392,NaN,GB-COH-04530979,Somerset Community Foundation,1
360G-SomersetCF-A405480,GBP,9962.00,2016-06-14,GB-COH-04983733,Tone Leisure (South West) Ltd,1110756,04983733,GB-COH-04530979,Somerset Community Foundation,1
360G-SomersetCF-A405664,GBP,11000.00,2016-06-14,GB-CHC-305633,"Blackdown District Scouts, Tangier Scout & Gui...",305633,NaN,GB-COH-04530979,Somerset Community Foundation,1
360G-SomersetCF-A406072,GBP,500.00,2016-06-30,360G-SomersetCF-ACC766126,Burnham United Junior Football Club,NaN,NaN,GB-COH-04530979,Somerset Community Foundation,1
360G-SomersetCF-A406345,GBP,500.00,2016-06-01,360G-SomersetCF-ACC350213,Good Fellowship Club,NaN,NaN,GB-COH-04530979,Somerset Community Foundation,1


### 2. Transform into a new format

I might want to filter the data before plotting it, so it's useful to copy to
a new variable to keep the original `grants` variable as is. 

In this case I've filtered to only include grants in 2017.

In [28]:
to_use = grants[grants["Award Date"].dt.year == 2017]
print("Using {:,.0f} grants".format(len(to_use)))

Using 31,290 grants


I create a table showing links between funders and recipients.
This has one row per funding relationship, from Funder > Recipient. 
I've added some columns showing the number of grants made by each 
funder to the recipient and the value of those grants.

To do this I use the `groupby` pandas function to group, and then
`sum` to pick up the amount and number of grants. I've reset the index
to make it easier to use the table later on.

In [30]:
links = to_use.groupby(
    ["Recipient Org:Identifier", "Funding Org:Identifier"]
).sum()[["Amount Awarded", "Grants"]].reset_index()

I only want to use links including recipients that have received
grants from more than one funder, to limit the size of the network
diagram. 

To do this I've used the `value_counts()` function to get a list
of unique recipients and how many relationships they occur in.

In [33]:
link_recipients = links["Recipient Org:Identifier"].value_counts()
print("{:,.0f} unique recipients".format(len(link_recipients)))

24,976 unique recipients


Then I filter this list to include only those links where the
recipient appears more than once.

In [34]:
links = links[links["Recipient Org:Identifier"].isin(link_recipients[link_recipients>1].index)]

We also add in the names of the funders to help with the data.

In [35]:
links = links.join(grants.groupby("Funding Org:Identifier").first()["Funding Org:Name"], on="Funding Org:Identifier")

In [36]:
print("{:,.0f} unique recipients with more than one funder in the period".format(len(links["Recipient Org:Identifier"].value_counts())))

2,316 unique recipients with more than one funder in the period


In [24]:
links

,Recipient Org:Identifier,Funding Org:Identifier,Amount Awarded,Grants,Funding Org:Name
7985,360G-trafford-theatre_of_the_senses,GB-COH-04831118,467.00,1,Trafford Housing Trust Social Investment
7986,360G-trafford-theatre_of_the_senses,GB-LAE-TRF,1720.00,1,Trafford Metropolitan Borough Council
8067,GB-CHC-1000011,360G-blf,8348.00,1,The Big Lottery Fund
8068,GB-CHC-1000011,GB-CHC-226446,39296.00,2,Seafarers UK
8075,GB-CHC-1000340,360G-blf,464600.00,1,The Big Lottery Fund
8076,GB-CHC-1000340,GB-CHC-230260,10000.00,1,Garfield Weston Foundation
8077,GB-CHC-1000351,360G-blf,10000.00,1,The Big Lottery Fund
8078,GB-CHC-1000351,GB-COH-IP00525R,3162.39,1,Co-operative Group
8092,GB-CHC-1000714,360G-blf,9920.00,1,The Big Lottery Fund
8093,GB-CHC-1000714,GB-CHC-230260,5000.00,1,Garfield Weston Foundation


### Create Chord output

The next step is to turn the links into a table with the number 
of recipients that pairs of funders have in common.

First I get a list of the funders in the dataset.

In [153]:
funders = links["Funding Org:Name"].unique()

Then go through each funder and get a list of the recipients of the funder,
and which other funders have funded them.

In [154]:
funder_rels = {}
# go through all the funders
for f in funders:
    
    # a list of the recipients of this funder
    recipients = links.loc[
                links["Funding Org:Name"]==f, 
                "Recipient Org:Identifier"].unique()
    
    # get a list of all the other funders that have funded those recipients
    funder_rels[f] = pd.DataFrame(links.loc[
        links["Recipient Org:Identifier"].isin(recipients) & (links["Funding Org:Name"]!=f),
        "Funding Org:Name"
    ].value_counts().rename("grants"))

Turn into a list of funder > funder relationships.

In [155]:
funder_rels = pd.concat(funder_rels, names=["Funder from", "Funder to"])

Save to a CSV file.

In [157]:
funder_rels.to_csv("chord.csv")

This CSV file is in the right format to use in a [Flourish chord diagram](https://app.flourish.studio/@flourish/chord-diagram).